In [38]:
import nltk
from itertools import groupby
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import re
import string

In [39]:
import pandas as pd

class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),s["POS"].values.tolist(),s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [105]:
data = pd.read_csv("../../data/ner_dataset.csv", encoding="latin1").fillna(method="ffill")
getter = SentenceGetter(data)
sent = getter.get_next()


In [106]:
sentences = getter.sentences
sentences[0]

[('Thousands', 'NNS', 'O'),
 ('of', 'IN', 'O'),
 ('demonstrators', 'NNS', 'O'),
 ('have', 'VBP', 'O'),
 ('marched', 'VBN', 'O'),
 ('through', 'IN', 'O'),
 ('London', 'NNP', 'B-geo'),
 ('to', 'TO', 'O'),
 ('protest', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('war', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('Iraq', 'NNP', 'B-geo'),
 ('and', 'CC', 'O'),
 ('demand', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('withdrawal', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('British', 'JJ', 'B-gpe'),
 ('troops', 'NNS', 'O'),
 ('from', 'IN', 'O'),
 ('that', 'DT', 'O'),
 ('country', 'NN', 'O'),
 ('.', '.', 'O')]

In [107]:
from sklearn.model_selection import train_test_split
X_train, X_test, __, __ = train_test_split(sentences,sentences,test_size=0.2, random_state=0)
X_valid, X_test, __, __ = train_test_split(X_test,X_test,test_size=0.5, random_state=0)

In [113]:
def extract_output(stc_tokens,path):
    with open(path,'w') as f:
        res = []
        for i in stc_tokens:
            stc=''
            for j in i:
                if j[-1]!='':
                    stc+=(' '.join(j)+'\n')
            res.append(stc)
        f.write(('\n'.join(res)).strip())

In [114]:
for i in X_valid:
    stc=''
    for j in i:
        if j[-1]=='':
            print('find')

In [115]:
extract_output(X_train,'../../data/train.txt')
extract_output(X_test,'../../data/test.txt')
extract_output(X_valid,'../../data/valid.txt')

In [89]:
X_train[0]

[('The', 'DT', 'O'),
 ('Iraqi', 'JJ', 'B-gpe'),
 ('government', 'NN', 'O'),
 ('is', 'VBZ', 'O'),
 ('urging', 'VBG', 'O'),
 ('voters', 'NNS', 'O'),
 ('to', 'TO', 'O'),
 ('approve', 'VB', 'O'),
 ('a', 'DT', 'O'),
 ('new', 'JJ', 'O'),
 ('constitution', 'NN', 'O'),
 ('Saturday', 'NNP', 'B-tim'),
 (',', ',', 'O'),
 ('and', 'CC', 'O'),
 ('says', 'VBZ', 'O'),
 ('Iraqis', 'NNS', 'B-gpe'),
 ('should', 'MD', 'O'),
 ('stand', 'VB', 'O'),
 ('firm', 'NN', 'O'),
 ('against', 'IN', 'O'),
 ('insurgents', 'NNS', 'O'),
 ('trying', 'VBG', 'O'),
 ('to', 'TO', 'O'),
 ('stop', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('referendum', 'NN', 'O'),
 ('.', '.', 'O')]

In [90]:
X_test[0]

[('Egypt', 'NNP', 'B-geo'),
 ('has', 'VBZ', 'O'),
 ('kept', 'VBN', 'O'),
 ('the', 'DT', 'O'),
 ('Rafah', 'NN', 'B-org'),
 ('crossing', 'VBG', 'O'),
 ('closed', 'JJ', 'O'),
 ('since', 'IN', 'O'),
 ('the', 'DT', 'O'),
 ('militant', 'JJ', 'O'),
 ('group', 'NN', 'O'),
 ('Hamas', 'NNP', 'B-org'),
 ('seized', 'VBD', 'O'),
 ('control', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('Gaza', 'NNP', 'B-geo'),
 ('more', 'JJR', 'O'),
 ('than', 'IN', 'O'),
 ('a', 'DT', 'O'),
 ('year', 'NN', 'O'),
 ('ago', 'RB', 'O'),
 ('.', '.', 'O')]

In [68]:
def is_number(string):
    pattern = r'^[+-]?\d*\.?\d+$'
    return bool(re.match(pattern, string))

In [69]:
prep_dict=['above', 'across', 'against', 'along', 'among', 'around', 'at', 'before', 'behind', 'below', 'beneath', 'beside', 'between', 'by', 'down', 'from', 'in', 'into', 'near', 'of', 'off', 'on', 'to', 'toward', 'under', 'upon', 'with', 'within']

In [70]:
stemmer = SnowballStemmer("english")
lm = WordNetLemmatizer()
train_feature=[]
for stc in X_train:
    temp_stc=[]
    for i in range(len(stc)):
        temp=[]
        temp.append(stc[i][0])
        temp.append('POS='+stc[i][1])
        if(i-1<0):
            temp.append('previous_1_word=Beg_Stc')
            temp.append('previous_1_POS=Begin_Sent')
            temp.append('previous_1_isCap=0')
            temp.append('Previous_BIO=')
        else:
            temp.append('previous_1_word='+stc[i-1][0])
            temp.append('previous_1_POS='+stc[i-1][1])
            temp.append('Previous_BIO='+stc[i-1][2])
            if (stc[i-1][0][0].isupper()):
                temp.append('previous_1_isCap=1')
            else:
                temp.append('previous_1_isCap=0')
        if(i-2<0):
            temp.append('previous_2_word=Beg_Stc')
            temp.append('previous_2_POS=Begin_Sent')
            temp.append('previous_2_isCap=0')
        else:
            temp.append('previous_2_word='+stc[i-2][0])
            temp.append('previous_2_POS='+stc[i-2][1])
            if (stc[i-2][0][0].isupper()):
                temp.append('previous_2_isCap=1')
            else:
                temp.append('previous_2_isCap=0')
        if(i+1>len(stc)-1):
            temp.append('next_1_word=End_Stc')
            temp.append('next_1_POS=End_Sent')
            temp.append('next_1_isCap=0')
        else:
            temp.append('next_1_word='+stc[i+1][0])
            temp.append('next_1_POS='+stc[i+1][1])
            if (stc[i+1][0][0].isupper()):
                temp.append('next_1_isCap=1')
            else:
                temp.append('next_1_isCap=0')
        if(i+2>len(stc)-1):
            temp.append('next_2_word=End_Stc')
            temp.append('next_2_POS=End_Sent')
            temp.append('next_2_isCap=0')
        else:
            temp.append('next_2_word='+stc[i+2][0])
            temp.append('next_2_POS='+stc[i+2][1])
            if (stc[i+2][0][0].isupper()):
                temp.append('next_2_isCap=1')
            else:
                temp.append('next_2_isCap=0')
        if (stc[i][0][0].isupper()):
            temp.append('isCap=1')
        else:
            temp.append('isCap=0')
        # temp.append('stemmed='+stemmer.stem(stc[i][0]))
        # temp.append('lemmatized='+lm.lemmatize(stc[i][0]))
        # # if(stc[i-1][2]!='O' and stc[i][2]!='O'):
        # #     temp.append('Previous_BIO=@@')
        # temp.append('position='+str((i+1)))
        # temp.append('stc_len='+str(len(stc)))
        temp.append('last_char='+stc[i][0][-1])
        # temp.append('length='+str(len(stc[i][0])))   
        temp.append('isNumber='+str(int(is_number(stc[i][0]))))     
        temp.append('isPunc='+str(int(stc[i][0] in string.punctuation)))
        temp.append('isAbbr='+str(int(stc[i][0][-1] == '.' )))
        # temp.append('isTheWordTo='+str(int(stc[i][0].lower() == 'to')))
        temp.append('isTheWordPrep='+str(int(stc[i][0].lower() in prep_dict)))
        # temp.append('Previous_BIO=@@')
        temp.append(stc[i][2])
        temp_stc.append(temp)
    train_feature.append(temp_stc)

In [71]:
train_feature[:5]

[[['The',
   'POS=DT',
   'previous_1_word=Beg_Stc',
   'previous_1_POS=Begin_Sent',
   'previous_1_isCap=0',
   'Previous_BIO=',
   'previous_2_word=Beg_Stc',
   'previous_2_POS=Begin_Sent',
   'previous_2_isCap=0',
   'next_1_word=Iraqi',
   'next_1_POS=JJ',
   'next_1_isCap=1',
   'next_2_word=government',
   'next_2_POS=NN',
   'next_2_isCap=0',
   'isCap=1',
   'last_char=e',
   'isNumber=0',
   'isPunc=0',
   'isAbbr=0',
   'isTheWordPrep=0',
   'O'],
  ['Iraqi',
   'POS=JJ',
   'previous_1_word=The',
   'previous_1_POS=DT',
   'Previous_BIO=O',
   'previous_1_isCap=1',
   'previous_2_word=Beg_Stc',
   'previous_2_POS=Begin_Sent',
   'previous_2_isCap=0',
   'next_1_word=government',
   'next_1_POS=NN',
   'next_1_isCap=0',
   'next_2_word=is',
   'next_2_POS=VBZ',
   'next_2_isCap=0',
   'isCap=1',
   'last_char=i',
   'isNumber=0',
   'isPunc=0',
   'isAbbr=0',
   'isTheWordPrep=0',
   'B-gpe'],
  ['government',
   'POS=NN',
   'previous_1_word=Iraqi',
   'previous_1_POS=JJ',
 

In [72]:
test_feature=[]
for stc in X_test:
    temp_stc=[]
    for i in range(len(stc)):
        temp=[]
        temp.append(stc[i][0])
        temp.append('POS='+stc[i][1])
        if(i-1<0):
            temp.append('previous_1_word=Beg_Stc')
            temp.append('previous_1_POS=Begin_Sent')
            temp.append('previous_1_isCap=0')
            temp.append('Previous_BIO=')
        else:
            temp.append('previous_1_word='+stc[i-1][0])
            temp.append('previous_1_POS='+stc[i-1][1])
            temp.append('Previous_BIO=@@')
            if (stc[i-1][0][0].isupper()):
                temp.append('previous_1_isCap=1')
            else:
                temp.append('previous_1_isCap=0')
        if(i-2<0):
            temp.append('previous_2_word=Beg_Stc')
            temp.append('previous_2_POS=Begin_Sent')
            temp.append('previous_2_isCap=0')
        else:
            temp.append('previous_2_word='+stc[i-2][0])
            temp.append('previous_2_POS='+stc[i-2][1])
            if (stc[i-2][0][0].isupper()):
                temp.append('previous_2_isCap=1')
            else:
                temp.append('previous_2_isCap=0')
        if(i+1>len(stc)-1):
            temp.append('next_1_word=End_Stc')
            temp.append('next_1_POS=End_Sent')
            temp.append('next_1_isCap=0')
        else:
            temp.append('next_1_word='+stc[i+1][0])
            temp.append('next_1_POS='+stc[i+1][1])
            if (stc[i+1][0][0].isupper()):
                temp.append('next_1_isCap=1')
            else:
                temp.append('next_1_isCap=0')
        if(i+2>len(stc)-1):
            temp.append('next_2_word=End_Stc')
            temp.append('next_2_POS=End_Sent')
            temp.append('next_2_isCap=0')
        else:
            temp.append('next_2_word='+stc[i+2][0])
            temp.append('next_2_POS='+stc[i+2][1])
            if (stc[i+2][0][0].isupper()):
                temp.append('next_2_isCap=1')
            else:
                temp.append('next_2_isCap=0')
        if (stc[i][0][0].isupper()):
            temp.append('isCap=1')
        else:
            temp.append('isCap=0')
        # temp.append('stemmed='+stemmer.stem(stc[i][0]))
        # temp.append('lemmatized='+lm.lemmatize(stc[i][0]))
        # # if(stc[i-1][2]!='O' and stc[i][2]!='O'):
        # #     temp.append('Previous_BIO=@@')
        # temp.append('position='+str((i+1)))
        # temp.append('stc_len='+str(len(stc)))
        temp.append('last_char='+stc[i][0][-1])
        # temp.append('length='+str(len(stc[i][0])))   
        temp.append('isNumber='+str(int(is_number(stc[i][0]))))     
        temp.append('isPunc='+str(int(stc[i][0] in string.punctuation)))
        temp.append('isAbbr='+str(int(stc[i][0][-1] == '.' )))
        # temp.append('isTheWordTo='+str(int(stc[i][0].lower() == 'to')))
        temp.append('isTheWordPrep='+str(int(stc[i][0].lower() in prep_dict)))
        # temp.append(stc[i][2])
        temp_stc.append(temp)
    test_feature.append(temp_stc)

In [73]:
test_feature[:5]

[[['Egypt',
   'POS=NNP',
   'previous_1_word=Beg_Stc',
   'previous_1_POS=Begin_Sent',
   'previous_1_isCap=0',
   'Previous_BIO=',
   'previous_2_word=Beg_Stc',
   'previous_2_POS=Begin_Sent',
   'previous_2_isCap=0',
   'next_1_word=has',
   'next_1_POS=VBZ',
   'next_1_isCap=0',
   'next_2_word=kept',
   'next_2_POS=VBN',
   'next_2_isCap=0',
   'isCap=1',
   'last_char=t',
   'isNumber=0',
   'isPunc=0',
   'isAbbr=0',
   'isTheWordPrep=0'],
  ['has',
   'POS=VBZ',
   'previous_1_word=Egypt',
   'previous_1_POS=NNP',
   'Previous_BIO=@@',
   'previous_1_isCap=1',
   'previous_2_word=Beg_Stc',
   'previous_2_POS=Begin_Sent',
   'previous_2_isCap=0',
   'next_1_word=kept',
   'next_1_POS=VBN',
   'next_1_isCap=0',
   'next_2_word=the',
   'next_2_POS=DT',
   'next_2_isCap=0',
   'isCap=0',
   'last_char=s',
   'isNumber=0',
   'isPunc=0',
   'isAbbr=0',
   'isTheWordPrep=0'],
  ['kept',
   'POS=VBN',
   'previous_1_word=has',
   'previous_1_POS=VBZ',
   'Previous_BIO=@@',
   'previo

In [74]:
def extract_output(stc_tokens,path):
    with open(path,'w') as f:
        f.write('\n')
        for i in stc_tokens:
            for j in i:
                f.write('\t'.join(j)+'\n')
            f.write('\n')

In [75]:
extract_output(train_feature,'training.feature')
extract_output(test_feature,'test.feature')

In [76]:
!java -cp .:maxent-3.0.0.jar:trove.jar MEtrain training.feature model.chunk 

Indexing events using cutoff of 4

	Computing event counts...  done. 839368 events
	Indexing...  done.
Sorting and merging events... done. Reduced 839368 events to 758060.
Done indexing.
Incorporating indexed data for training...  
done.
	Number of Event Tokens: 758060
	    Number of Outcomes: 17
	  Number of Predicates: 54567
...done.
Computing model parameters...
Performing 100 iterations.
  1:  .. loglikelihood=-2378108.6181685124	0.8472862915908159
  2:  .. loglikelihood=-473639.84763823234	0.8472862915908159
  3:  .. loglikelihood=-358002.89683618926	0.8478855519867329
  4:  .. loglikelihood=-285470.21143335814	0.8691038972179068
  5:  .. loglikelihood=-238984.97525349713	0.89934331544686
  6:  .. loglikelihood=-207461.7031756231	0.923806959521926
  7:  .. loglikelihood=-184918.89674834025	0.9390970349119814
  8:  .. loglikelihood=-168075.064614489	0.9485577243831073
  9:  .. loglikelihood=-155029.2859025349	0.9546897189313864
 10:  .. loglikelihood=-144621.84436175576	0.958630779

In [77]:
!java -cp .:maxent-3.0.0.jar:trove.jar MEtag test.feature model.chunk ../../output/maxent.txt

In [78]:
from seqeval.metrics import classification_report

In [79]:
with open('../../output/maxent.txt','r') as f:
    text=f.read()
    train_tokens=list(x.split('\t') for x in text.split('\n')[:-1])
pred_maxent=[list(group) for k, group in groupby(train_tokens, lambda x: x == [""]) if not k]

In [80]:
pred_maxent=[[i[-1] for i in x] for x in pred_maxent]

In [81]:
pred_maxent[0]

['B-org',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-org',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [82]:
true_label = [[i[-1] for i in x] for x in X_test]
true_label[0]

['B-geo',
 'O',
 'O',
 'O',
 'B-org',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-org',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [116]:
print(classification_report(true_label, pred_maxent))

              precision    recall  f1-score   support

         art       0.40      0.09      0.15        45
         eve       0.58      0.37      0.45        30
         geo       0.81      0.89      0.85      3763
         gpe       0.95      0.90      0.93      1588
         nat       0.91      0.31      0.47        32
         org       0.69      0.67      0.68      1980
         per       0.73      0.71      0.72      1681
         tim       0.86      0.77      0.81      2139

   micro avg       0.81      0.80      0.80     11258
   macro avg       0.74      0.59      0.63     11258
weighted avg       0.80      0.80      0.80     11258



In [117]:
print(classification_report(pred_maxent, true_label))

              precision    recall  f1-score   support

         art       0.09      0.40      0.15        10
         eve       0.37      0.58      0.45        19
         geo       0.89      0.81      0.85      4140
         gpe       0.90      0.95      0.93      1511
         nat       0.31      0.91      0.47        11
         org       0.67      0.69      0.68      1895
         per       0.71      0.73      0.72      1633
         tim       0.77      0.86      0.81      1928

   micro avg       0.80      0.81      0.80     11147
   macro avg       0.59      0.74      0.63     11147
weighted avg       0.80      0.81      0.80     11147

